In [6]:
import os
import flask
import vanna
from vanna.remote import VannaDefault
from dotenv import load_dotenv
from vanna.flask import VannaFlaskApp

In [7]:
load_dotenv()
api_key = os.getenv('VANNA_API_KEY')
vanna_model_name = os.getenv('VANNA_MODEL')
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

Python-dotenv could not parse statement starting at line 9


In [8]:
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT", "5432")  # default to 5432 if not set

vn.connect_to_postgres(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)


In [11]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn,title="FreshBus AI ChatBot",allow_llm_to_see_data=False,sql=False,csv_download=False)
app.run()

Python-dotenv could not parse statement starting at line 9


Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
